In [1]:
import pandas as pd
import numpy as np

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

/nfs/home/ababjac/base-venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]
2023-01-23 09:47:59.597368: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 09:47:59.726181: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-23 09:47:59.733857: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /sw/isaac/compilers/intel/oneAPI_2021.2.0/mpi/latest/libfabric/lib:/sw/isaac/

In [2]:
PATH='/lustre/isaac/proj/UTK0196/deep-surface-protein-data/'

In [3]:
df = pd.read_csv(PATH+'M0059E_training_set.tsv', delimiter=',', nrows=5000, header=0)

In [4]:
df.columns

Index(['deep.ID', 'surf.ID', 'deep.sequence', 'surf.sequence',
       'percent.identity', 'alignment.length', 'bitscore'],
      dtype='object')

In [5]:
df.shape

(5000, 7)

In [6]:
df

,deep.ID,surf.ID,deep.sequence,surf.sequence,percent.identity,alignment.length,bitscore
0,SRR7066492_k141_369890_2,SRR7066493_k141_874768_2,MTENERKFTLVGLGEILWDVLPDGKQLGGAPANFAYHAQALGGRGI...,MTERGKYVCVGLGEILWDMLPEGKQLGGAPANFAYHAQALRGQGVV...,70.7,256,360.1
1,SRR7066492_k141_369890_2,SRR7066493_k141_1284874_2,MTENERKFTLVGLGEILWDVLPDGKQLGGAPANFAYHAQALGGRGI...,MTVDGKYLCVGLGEILWDMLPGGKQLGGAPANFAYHSQALGAQGVV...,70.4,250,355.1
2,SRR7066492_k141_443867_2,SRR7066493_k141_1612491_4,MKVALLGLLQSGKSSIFAGLSGKSIPPVGSTAIEEAIVPVPDERLD...,MKVALVGLLQSGKSTILASLSGKAIPAIGSASIEEAIVSVPDDRFD...,78.4,356,559.3
3,SRR7066492_k141_443867_2,SRR7066493_k141_1619980_3,MKVALLGLLQSGKSSIFAGLSGKSIPPVGSTAIEEAIVPVPDERLD...,MKVALIGLLQSGKSTILASLTGKAIPAIGSASIEETIVPVPDERFD...,78.1,356,555.1
4,SRR7066492_k141_443867_2,SRR7066493_k141_1671894_1,MKVALLGLLQSGKSSIFAGLSGKSIPPVGSTAIEEAIVPVPDERLD...,MKVALIGLLQSGKSTILASLTGKAVPAAGSASIEEAIVPVPDERFD...,77.5,356,550.4
...,...,...,...,...,...,...,...
4995,SRR7066492_k141_666341_2,SRR7066493_k141_242132_1,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,MGQKEGVILIVDDERDHADGLAESLEKLCARAIAVYDGTDALQILR...,75.1,370,548.1
4996,SRR7066492_k141_666341_2,SRR7066493_k141_1618303_2,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,MAQKAGVILVVDDERDHADGIVESLEKLCTQAIAVYNGTDALEIVR...,74.7,367,538.1
4997,SRR7066492_k141_666341_2,SRR7066493_k141_1439149_3,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,MKQKANIILVVDDERDHADGIAEALEKLCTKAIAVYTGKDALEIVR...,73.8,370,535.4
4998,SRR7066492_k141_666341_2,SRR7066493_k141_1723797_3,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,MAQKAGVILVVDDERDHADGIVESLEKLCTRAIAVYNGTDAMEIVR...,74.7,367,532.7


In [7]:
surf_series = df['surf.sequence']
deep_series = df['deep.sequence']

classification_df = pd.DataFrame({'text' : surf_series.append(deep_series, ignore_index=True), 'label' : [0]*surf_series.size+[1]*deep_series.size})
classification_df

,text,label
0,MTERGKYVCVGLGEILWDMLPEGKQLGGAPANFAYHAQALRGQGVV...,0
1,MTVDGKYLCVGLGEILWDMLPGGKQLGGAPANFAYHSQALGAQGVV...,0
2,MKVALVGLLQSGKSTILASLSGKAIPAIGSASIEEAIVSVPDDRFD...,0
3,MKVALIGLLQSGKSTILASLTGKAIPAIGSASIEETIVPVPDERFD...,0
4,MKVALIGLLQSGKSTILASLTGKAVPAAGSASIEEAIVPVPDERFD...,0
...,...,...
9995,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,1
9996,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,1
9997,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,1
9998,MDKDTVILVVDDEREHADGIAEAMEKLCGRAIAVYNGADALEIVRN...,1


In [8]:
def overlap_sequence(seq, word_length, overlap):
    if overlap >= word_length:
        print('Overlap must be less than word length')
        return
    
    for i in range(0, len(seq)-overlap, word_length-overlap):
        yield seq[i:i+word_length]
        
def get_overlap_array(seq, word_length=5, overlap=2):
    return np.array(list(overlap_sequence(seq, word_length, overlap)))

def get_overlap_string(seq, word_length=5, overlap=2):
    return ' '.join(list(overlap_sequence(seq, word_length, overlap)))

In [9]:
classification_df['text'] = classification_df['text'].transform(get_overlap_string)
classification_df

,text,label
0,MTERG RGKYV YVCVG VGLGE GEILW LWDML MLPEG EGKQ...,0
1,MTVDG DGKYL YLCVG VGLGE GEILW LWDML MLPGG GGKQ...,0
2,MKVAL ALVGL GLLQS QSGKS KSTIL ILASL SLSGK GKAI...,0
3,MKVAL ALIGL GLLQS QSGKS KSTIL ILASL SLTGK GKAI...,0
4,MKVAL ALIGL GLLQS QSGKS KSTIL ILASL SLTGK GKAV...,0
...,...,...
9995,MDKDT DTVIL ILVVD VDDER EREHA HADGI GIAEA EAME...,1
9996,MDKDT DTVIL ILVVD VDDER EREHA HADGI GIAEA EAME...,1
9997,MDKDT DTVIL ILVVD VDDER EREHA HADGI GIAEA EAME...,1
9998,MDKDT DTVIL ILVVD VDDER EREHA HADGI GIAEA EAME...,1


In [10]:
ds = Dataset.from_pandas(classification_df)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
tokenized_ds = ds.map(lambda d : tokenizer(d['text'], truncation=True), batched=True)

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  8.65ba/s]


In [13]:
init_splits = tokenized_ds.train_test_split(test_size=0.2)

tmp = init_splits['train']
test_ds = init_splits['test']

splits = tmp.train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

In [16]:
training_args = TrainingArguments(
    output_dir='./train_output',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    #data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/nfs/home/ababjac/base-venv/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7200
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2250
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encod

Step,Training Loss
500,0.694800
1000,0.684400
1500,0.586900
2000,0.488900


Saving model checkpoint to ./train_output/checkpoint-500
Configuration saved in ./train_output/checkpoint-500/config.json
Model weights saved in ./train_output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./train_output/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./train_output/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./train_output/checkpoint-1000
Configuration saved in ./train_output/checkpoint-1000/config.json
Model weights saved in ./train_output/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./train_output/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./train_output/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./train_output/checkpoint-1500
Configuration saved in ./train_output/checkpoint-1500/config.json
Model weights saved in ./train_output/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./train_output/checkpoint-1500/tokenizer_config.js

TrainOutput(global_step=2250, training_loss=0.5952728949652778, metrics={'train_runtime': 9395.4305, 'train_samples_per_second': 3.832, 'train_steps_per_second': 0.239, 'total_flos': 4514998117760064.0, 'train_loss': 0.5952728949652778, 'epoch': 5.0})

In [19]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16


{'eval_loss': 0.6573528051376343,
 'eval_runtime': 52.1002,
 'eval_samples_per_second': 38.388,
 'eval_steps_per_second': 2.399,
 'epoch': 5.0}

In [20]:
trainer.predict(test_dataset=test_ds)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 16


PredictionOutput(predictions=array([[ 0.82968235, -0.6178355 ],
       [-1.5641358 ,  1.6981683 ],
       [-1.7991769 ,  1.9067947 ],
       ...,
       [ 0.46855587, -0.23162535],
       [ 0.81024885, -0.6906252 ],
       [ 1.0147853 , -0.7247598 ]], dtype=float32), label_ids=array([0, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.6573528051376343, 'test_runtime': 42.1754, 'test_samples_per_second': 47.421, 'test_steps_per_second': 2.964})

In [ ]:
trainer.save_pretrained('./models/initial')